# DecoyComp: Dataset Comparison Executive Summary

**Purpose**: Quick comparison of molecular benchmark datasets for virtual screening

**Datasets Analyzed**: LIT-PCBA, DUDE-Z, DEKOIS2, MUV, D-COID

---

## Prerequisites

Before running this notebook, ensure you have generated the dataset summaries:

```bash
# Generate file-based dataset summaries
python analyse_datasets.py --roots LIT-PCBA DUDE-Z DEKOIS2 D-COID --include-muv --outdir results

# Generate visualizations (optional, can be done in notebook)
python metrics.py --smiles-dir results/smiles --outdir results --summary-csv results/dataset_unique_summary_split.csv
```

## 1. Setup & Data Loading

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Import functions from metrics.py
from metrics import COLOR_ACTIVE, COLOR_INACTIVE, violin_plot, compliance_bar_from_summary

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = (10, 6)

# Configuration
RESULTS_DIR = Path('results')
SMILES_DIR = RESULTS_DIR / 'smiles'
OUTPUT_DIR = Path('notebook_figures')
OUTPUT_DIR.mkdir(exist_ok=True)

# Dataset colors (consistent throughout notebook)
DATASET_COLORS = {
    'LIT-PCBA': '#1f77b4',
    'DUDE-Z': '#ff7f0e',
    'DEKOIS2': '#2ca02c',
    'MUV': '#d62728',
    'D-COID': '#9467bd'
}

print("✓ Setup complete")

In [ ]:
# Load pre-computed summaries
summary_file = RESULTS_DIR / 'dataset_unique_summary.csv'
split_summary_file = RESULTS_DIR / 'dataset_unique_summary_split.csv'

if not summary_file.exists():
    raise FileNotFoundError(
        f"Summary file not found: {summary_file}\n"
        "Please run: python analyse_datasets.py --roots LIT-PCBA DUDE-Z DEKOIS2 D-COID --include-muv --outdir results"
    )

# Load data
df_summary = pd.read_csv(summary_file)
df_split = pd.read_csv(split_summary_file) if split_summary_file.exists() else None

print(f"✓ Loaded summary data:")
print(f"  - Overall summary: {df_summary.shape}")
if df_split is not None:
    print(f"  - Split summary: {df_split.shape}")

# Display available datasets
datasets = df_summary['Dataset'].unique() if 'Dataset' in df_summary.columns else []
print(f"\n✓ Datasets found: {', '.join(datasets)}")

## 2. Quick Overview: Dataset Comparison Table

In [ ]:
# Create comprehensive comparison table
def create_comparison_table(df_summary, df_split):
    """Create a summary comparison table for all datasets."""
    
    comparison = []
    
    for dataset in df_summary['Dataset'].unique():
        # Get overall stats
        overall = df_summary[df_summary['Dataset'] == dataset].iloc[0]
        
        # Get actives and inactives counts from overall summary
        n_actives = int(overall.get('NumberActivesUnique', 0))
        n_inactives = int(overall.get('NumberInactivesUnique', 0))
        total_mols = overall['NumberLigandsUnique']
        
        # Calculate quality score (100 - percentage of problematic molecules)
        invalid_pct = (overall.get('NumberInvalidSMILES', 0) / total_mols * 100) if total_mols > 0 else 0
        quality_score = 100 - invalid_pct
        
        # Drug-likeness
        lipinski = overall.get('LipinskiComplianceRate', 0) * 100
        veber = overall.get('VeberComplianceRate', 0) * 100
        
        comparison.append({
            'Dataset': dataset,
            'Total Molecules': f"{int(total_mols):,}",
            'Actives': f"{n_actives:,}",
            'Decoys/Inactives': f"{n_inactives:,}",
            'Active:Decoy Ratio': f"1:{n_inactives/n_actives:.1f}" if n_actives > 0 else 'N/A',
            'Quality Score': f"{quality_score:.1f}%",
            'Lipinski Compliant': f"{lipinski:.1f}%",
            'Veber Compliant': f"{veber:.1f}%"
        })
    
    return pd.DataFrame(comparison)

# Create and display comparison table
comparison_df = create_comparison_table(df_summary, df_split)

# Style the table
styled_table = comparison_df.style.set_properties(**{
    'text-align': 'left',
    'font-size': '11pt'
}).set_table_styles([{
    'selector': 'th',
    'props': [('font-size', '12pt'), ('text-align', 'center'), ('font-weight', 'bold')]
}])

display(styled_table)

## 3. Visual Summary

### 3.1 Dataset Size Comparison

In [ ]:
# Bar chart: Total molecules and actives/decoys breakdown
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Total molecules
datasets = df_summary['Dataset'].values
total_mols = df_summary['NumberLigandsUnique'].values
colors = [DATASET_COLORS.get(ds, 'gray') for ds in datasets]

ax1.bar(datasets, total_mols, color=colors, alpha=0.7)
ax1.set_ylabel('Number of Unique Molecules')
ax1.set_title('Dataset Sizes', fontweight='bold', fontsize=12)
ax1.tick_params(axis='x', rotation=45)

# Add value labels on bars
for i, (ds, val) in enumerate(zip(datasets, total_mols)):
    ax1.text(i, val, f'{int(val):,}', ha='center', va='bottom', fontsize=9)

# Actives vs Decoys breakdown
actives_counts = df_summary['NumberActivesUnique'].values
decoys_counts = df_summary['NumberInactivesUnique'].values

x = np.arange(len(datasets))
width = 0.35

ax2.bar(x - width/2, actives_counts, width, label='Actives', color=COLOR_ACTIVE, alpha=0.7)
ax2.bar(x + width/2, decoys_counts, width, label='Decoys/Inactives', color=COLOR_INACTIVE, alpha=0.7)

ax2.set_ylabel('Number of Molecules')
ax2.set_title('Active vs Decoy Distribution', fontweight='bold', fontsize=12)
ax2.set_xticks(x)
ax2.set_xticklabels(datasets, rotation=45, ha='right')
ax2.legend()

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'dataset_sizes.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Dataset size comparison complete")

### 3.2 Chemical Property Distributions

In [ ]:
# Load or compute molecular descriptors
descriptors_file = RESULTS_DIR / 'ligand_descriptors.csv'

if descriptors_file.exists():
    print(f"Loading descriptors from {descriptors_file}...")
    df_desc = pd.read_csv(descriptors_file)
    print(f"✓ Loaded {len(df_desc):,} molecular descriptors")
else:
    print("⚠ Descriptor file not found. Run metrics.py first or compute descriptors below.")
    print("  python metrics.py --smiles-dir results/smiles --outdir results")
    df_desc = None

In [ ]:
# Create violin plots for key descriptors
if df_desc is not None:
    print("Generating molecular property distribution plots...")
    
    # Use existing violin_plot function from metrics.py
    violin_plot(df_desc, "mw", "Molecular Weight (Da)", 
                "Molecular Weight Distribution", OUTPUT_DIR / "violin_mw.png")
    
    violin_plot(df_desc, "tpsa", "TPSA (Ų)", 
                "Topological Polar Surface Area", OUTPUT_DIR / "violin_tpsa.png")
    
    violin_plot(df_desc, "rotbonds", "Rotatable Bonds", 
                "Rotatable Bonds Distribution", OUTPUT_DIR / "violin_rotbonds.png")
    
    print("✓ Violin plots saved to", OUTPUT_DIR)
    
    # Display the MW plot inline
    from IPython.display import Image, display
    display(Image(filename=OUTPUT_DIR / "violin_mw.png"))
else:
    print("⚠ Skipping violin plots (descriptors not available)")

### 3.3 Drug-likeness Assessment

In [ ]:
# Drug-likeness compliance (Lipinski & Veber)
fig, ax = plt.subplots(figsize=(10, 6))

datasets = df_summary['Dataset'].values
lipinski_rates = df_summary['LipinskiComplianceRate'].values * 100
veber_rates = df_summary['VeberComplianceRate'].values * 100

x = np.arange(len(datasets))
width = 0.35

bars1 = ax.bar(x - width/2, lipinski_rates, width, label="Lipinski's Rule of Five", 
               color='#3498db', alpha=0.8)
bars2 = ax.bar(x + width/2, veber_rates, width, label="Veber's Rule", 
               color='#e74c3c', alpha=0.8)

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%', ha='center', va='bottom', fontsize=9)

ax.set_ylabel('Compliance Rate (%)', fontsize=11)
ax.set_title('Drug-likeness Rule Compliance', fontweight='bold', fontsize=13)
ax.set_xticks(x)
ax.set_xticklabels(datasets, rotation=45, ha='right')
ax.set_ylim(0, 105)
ax.legend(fontsize=10)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'druglikeness_compliance.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Drug-likeness compliance chart complete")

### 3.4 Data Quality Scorecard

In [ ]:
# Create quality metrics matrix
def create_quality_matrix(df_summary):
    """Create a quality assessment matrix."""
    quality_data = []
    
    for ds in df_summary['Dataset'].values:
        row = df_summary[df_summary['Dataset'] == ds].iloc[0]
        total = row['NumberLigandsUnique']
        
        # Calculate percentages
        invalid_pct = (row.get('NumberInvalidSMILES', 0) / total * 100) if total > 0 else 0
        valid_pct = 100 - invalid_pct
        lipinski_pct = row.get('LipinskiComplianceRate', 0) * 100
        veber_pct = row.get('VeberComplianceRate', 0) * 100
        
        quality_data.append({
            'Dataset': ds,
            'Valid SMILES': valid_pct,
            'Lipinski Compliant': lipinski_pct,
            'Veber Compliant': veber_pct
        })
    
    return pd.DataFrame(quality_data)

quality_df = create_quality_matrix(df_summary)

# Create heatmap
fig, ax = plt.subplots(figsize=(10, 5))

# Prepare data for heatmap (transpose so datasets are columns)
heatmap_data = quality_df.set_index('Dataset').T

# Create heatmap
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='RdYlGn', 
            vmin=0, vmax=100, cbar_kws={'label': 'Percentage (%)'},
            linewidths=0.5, ax=ax)

ax.set_title('Data Quality Scorecard', fontweight='bold', fontsize=13, pad=15)
ax.set_xlabel('Dataset', fontsize=11)
ax.set_ylabel('Quality Metric', fontsize=11)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'quality_scorecard.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Quality scorecard complete")

### 3.5 Chemical Space Visualization (PCA)

In [ ]:
# PCA visualization of chemical space
if df_desc is not None:
    print("Computing PCA for chemical space visualization...")
    
    # Sample if too large (for performance)
    max_samples_per_dataset = 5000
    df_sampled = df_desc.groupby('dataset').apply(
        lambda x: x.sample(n=min(len(x), max_samples_per_dataset), random_state=42)
    ).reset_index(drop=True)
    
    print(f"  Using {len(df_sampled):,} molecules for PCA (sampled from {len(df_desc):,})")
    
    # Prepare descriptor matrix
    X = df_sampled[['mw', 'tpsa', 'rotbonds']].values
    
    # Standardize and apply PCA
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    # Create scatter plot
    fig, ax = plt.subplots(figsize=(10, 8))
    
    for ds in df_sampled['dataset'].unique():
        mask = df_sampled['dataset'] == ds
        ax.scatter(X_pca[mask, 0], X_pca[mask, 1], 
                  label=ds, alpha=0.4, s=10,
                  color=DATASET_COLORS.get(ds, 'gray'))
    
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', fontsize=11)
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)', fontsize=11)
    ax.set_title('Chemical Space Distribution (PCA)', fontweight='bold', fontsize=13)
    ax.legend(loc='best', framealpha=0.9)
    ax.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / 'chemical_space_pca.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✓ PCA complete (explained variance: {pca.explained_variance_ratio_.sum()*100:.1f}%)")
else:
    print("⚠ Skipping PCA (descriptors not available)")

## 4. Key Findings & Recommendations

In [ ]:
# Auto-generate key findings
def generate_key_findings(df_summary, df_split):
    """Automatically identify and report key findings."""
    findings = []
    
    # 1. Dataset sizes
    largest_ds = df_summary.loc[df_summary['NumberLigandsUnique'].idxmax()]
    smallest_ds = df_summary.loc[df_summary['NumberLigandsUnique'].idxmin()]
    
    findings.append(
        f"**Dataset Size**: {largest_ds['Dataset']} is the largest with "
        f"{int(largest_ds['NumberLigandsUnique']):,} unique molecules, while "
        f"{smallest_ds['Dataset']} is the smallest with {int(smallest_ds['NumberLigandsUnique']):,} molecules."
    )
    
    # 2. Drug-likeness
    best_lipinski = df_summary.loc[df_summary['LipinskiComplianceRate'].idxmax()]
    findings.append(
        f"**Drug-likeness**: {best_lipinski['Dataset']} has the highest Lipinski compliance at "
        f"{best_lipinski['LipinskiComplianceRate']*100:.1f}%, making it most suitable for drug-like compound screening."
    )
    
    # 3. Data quality
    df_summary_copy = df_summary.copy()
    df_summary_copy['invalid_rate'] = df_summary_copy['NumberInvalidSMILES'] / df_summary_copy['NumberLigandsUnique']
    cleanest_ds = df_summary_copy.loc[df_summary_copy['invalid_rate'].idxmin()]
    findings.append(
        f"**Data Quality**: {cleanest_ds['Dataset']} has the lowest rate of invalid SMILES "
        f"({cleanest_ds['invalid_rate']*100:.3f}%), indicating high data quality."
    )
    
    # 4. Active:Decoy ratios
    ratios = []
    for ds in df_summary['Dataset'].unique():
        row = df_summary[df_summary['Dataset'] == ds].iloc[0]
        n_actives = row['NumberActivesUnique']
        n_inactives = row['NumberInactivesUnique']
        if n_actives > 0:
            ratios.append((ds, n_inactives / n_actives))
    
    if ratios:
        most_balanced = min(ratios, key=lambda x: abs(x[1] - 1))
        findings.append(
            f"**Balance**: {most_balanced[0]} has the most balanced active:decoy ratio "
            f"(1:{most_balanced[1]:.1f}), ideal for unbiased model training."
        )
    
    return findings

findings = generate_key_findings(df_summary, df_split)

print("## 📊 Key Findings\n")
for i, finding in enumerate(findings, 1):
    print(f"{i}. {finding}\n")

In [ ]:
# Use case recommendations
recommendations = {
    'Method Development & Testing': [
        'Use the cleanest dataset with high-quality SMILES',
        'Recommended: Dataset with highest data quality score'
    ],
    'Realistic Benchmarking': [
        'Use literature-derived datasets for real-world relevance',
        'Recommended: LIT-PCBA (literature sources)'
    ],
    'Challenging Model Evaluation': [
        'Use datasets with matched/sophisticated decoys',
        'Recommended: DUDE-Z (property-matched decoys)'
    ],
    'Drug-like Compound Screening': [
        'Use dataset with highest Lipinski compliance',
        f'Recommended: {df_summary.loc[df_summary["LipinskiComplianceRate"].idxmax(), "Dataset"]}'
    ],
    'Structure-based Studies': [
        'Use datasets with structural information',
        'Recommended: D-COID (PDB-derived structures)'
    ],
    'Bioactivity Prediction': [
        'Use datasets with experimental bioactivity data',
        'Recommended: MUV (bioactivity database)'
    ]
}

print("## 🎯 Use Case Recommendations\n")
for use_case, details in recommendations.items():
    print(f"**{use_case}**")
    for detail in details:
        print(f"  • {detail}")
    print()

## 5. Summary Statistics Table

In [ ]:
# Compute summary statistics for molecular descriptors
if df_desc is not None:
    stats_summary = df_desc.groupby('dataset').agg({
        'mw': ['mean', 'std', 'median'],
        'tpsa': ['mean', 'std', 'median'],
        'rotbonds': ['mean', 'std', 'median']
    }).round(2)
    
    # Flatten column names
    stats_summary.columns = [f'{desc}_{stat}' for desc, stat in stats_summary.columns]
    
    print("## 📈 Molecular Descriptor Statistics\n")
    display(stats_summary)
    
    # Save to CSV
    stats_summary.to_csv(OUTPUT_DIR / 'descriptor_statistics.csv')
    print(f"\n✓ Statistics saved to {OUTPUT_DIR / 'descriptor_statistics.csv'}")
else:
    print("⚠ Descriptor statistics not available (descriptors not loaded)")

## 6. Conclusions

### Summary

This executive summary compared five benchmark molecular datasets for virtual screening:

- **LIT-PCBA**: Literature-derived actives and inactives
- **DUDE-Z**: Property-matched decoys for challenging evaluation
- **DEKOIS2**: ZINC-derived decoys with diverse scaffolds
- **MUV**: Bioactivity database with experimental data
- **D-COID**: Structure-based dataset from PDB complexes

### Next Steps

1. **Select appropriate dataset(s)** based on use case recommendations above
2. **Consider combining datasets** for more robust validation
3. **Validate findings** with your specific screening protocol
4. **Consult comprehensive analysis** notebook for detailed statistics

---

**For detailed analysis**: See `dataset_comparison_comprehensive.ipynb` (when available)

**Questions or feedback**: [Contact information]

In [ ]:
# Print summary of generated outputs
print("\n" + "="*60)
print("📁 OUTPUTS GENERATED")
print("="*60)
print(f"\nAll figures saved to: {OUTPUT_DIR.absolute()}\n")

output_files = list(OUTPUT_DIR.glob('*.png')) + list(OUTPUT_DIR.glob('*.csv'))
for f in sorted(output_files):
    print(f"  ✓ {f.name}")

print(f"\nTotal files: {len(output_files)}")
print("\n" + "="*60)